In [ ]:

3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
print(pd.__version__)

In [ ]:
train = pd.read_csv('../input/multiclass/train.csv')
test = pd.read_csv('../input/multiclass/test.csv')

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [ ]:
train.isna().mean()

In [ ]:
test.isna().mean()

In [ ]:
train.nunique()

In [ ]:
train.hist(['9', 'target'], figsize=(10, 10))
None

In [ ]:
test.hist(['9'], figsize=(10, 10))
None

In [ ]:
train.describe()

In [ ]:
train.describe()

In [ ]:
test.describe()

## changing missing values to means

In [ ]:
df_numeric = train.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = train[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        med = train[col].median()
        train[col] = train[col].fillna(med)
for col in numeric_cols:
    print(train[col].median())


In [ ]:
train.isna().mean()

In [ ]:
df_numeric = test.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = test[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        med = test[col].median()
        test[col] = test[col].fillna(med)

In [ ]:
test.isna().mean()

In [ ]:
#from imblearn.over_sampling import ADASYN

In [ ]:
## dropping lines from biggest class

In [ ]:
train = train.drop(train[train['target'] == 0].sample(frac=0.45).index)
train = train.drop(train[train['target'] == 2].sample(frac=0.2).index)

In [ ]:
train.hist(['9', 'target'], figsize=(10, 10))
None

## outliers processing

In [ ]:
import seaborn as sns
#sns.pairplot(train)
None

In [ ]:
train.boxplot()

In [ ]:
for n in ['0', '1', '2', '3', '4', '5', '6', '7', '8']:
    for x in n:
        q75,q25 = np.percentile(train.loc[:,x],[75,25])
        intr_qr = q75-q25

        max = q75+(2*intr_qr)
        min = q25-(2*intr_qr)
        
        
        train.loc[train[x] < min,x] = train[n].quantile(0.25)
        
        train.loc[train[x] > max,x] = train[n].quantile(0.75)

In [ ]:
train.isnull().sum()

In [ ]:
for n in ['0', '1', '2', '3', '4', '5', '6', '7', '8']:
    for x in n:
        q75,q25 = np.percentile(test.loc[:,x],[75,25])
        intr_qr = q75-q25

        max = q75+(2*intr_qr)
        min = q25-(2*intr_qr)
        
        test.loc[test[x] < min,x] = np.percentile(test.loc[:,x],[25])
        test.loc[test[x] > max,x] = np.percentile(test.loc[:,x],[75])
        


In [ ]:
train_9 = pd.get_dummies(train['9'])
test_9 = pd.get_dummies(test['9'])

train.drop(columns = '9')
test.drop(columns = '9')

train = pd.concat([train,train_9], axis=1)
test = pd.concat([test,test_9], axis=1)
test

In [ ]:
X = train.drop(columns='target')

X['9'] = X['9'].astype(np.int64)
X.head()


In [ ]:
#from imblearn.over_sampling import SMOTE

In [ ]:
test['9'] = test['9'].astype(np.int64)

In [ ]:
y = train['target']
y.shape


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

In [ ]:
X_train

In [ ]:
y_validation

In [ ]:
 from catboost import Pool, CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import xgboost as xgb

from sklearn import datasets

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder 

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

In [ ]:
def macro_f1_score(y_true, y_pred, n_labels=5):
    total_f1 = 0.
    for i in range(n_labels):
        yt = y_true == i
        yp = y_pred == i

        tp = np.sum(yt & yp)

        tpfp = np.sum(yp)
        tpfn = np.sum(yt)
        if tpfp == 0:
            print('[WARNING] F-score is ill-defined and being set to 0.0 in labels with no predicted samples.')
            precision = 0.
        else:
            precision = tp / tpfp
        if tpfn == 0:
            print(f'[ERROR] label not found in y_true...')
            recall = 0.
        else:
            recall = tp / tpfn

        if precision == 0. or recall == 0.:
            f1 = 0.
        else:
            f1 = 2 * precision * recall / (precision + recall)
        total_f1 += f1
    return total_f1 / n_labels

In [ ]:
from sklearn.metrics import make_scorer
my_scorer = make_scorer(macro_f1_score, greater_is_better=True)



In [ ]:
from sklearn.metrics import f1_score
import numpy as np
    
def f1(y_pred, dtrain):
      y_true = dtrain.get_label()
      err = 1-f1_score(y_true, np.round(y_pred), average = 'macro')
      return 'f1_err', err


In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
clf = XGBClassifier(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective='multi:softmax',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
    feval = macro_f1_score,
        tree_method='gpu_hist')

In [ ]:
clf.fit(X_train, y_train,
        eval_set=[(X_validation,y_validation)],
        
        early_stopping_rounds = 50)

In [ ]:
ans = clf.predict(test)

In [ ]:
len(ans)

In [ ]:
ans1 = pd.DataFrame(ans, columns = ['target'])
ans1.to_csv('ansxgb2', encoding='utf-8', index=False)